In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
import pandas as pd
import time
import os
from torch.utils.data import DataLoader,Dataset,TensorDataset

In [13]:
# 参数设置
batch_size = 512
epochs = 20
L = 5
target_len = 3
neg_num = 3

In [3]:
# 读取数据
path_train = os.path.join(os.path.abspath('.'), 'datasets','ml1m/test/train.txt')
path_test = os.path.join(os.path.abspath('.'), 'datasets','ml1m/test/test.txt')
train = pd.read_csv(path_train, sep=' ', names=['user', 'item', 'rating'])
test = pd.read_csv(path_test, sep=' ', names=['user', 'item', 'rating'])

# map user and item uuid to id
user2id = {}
item2id = {}
for idx,user in enumerate(train['user'].unique().tolist()):
    user2id[user] = idx
for idx,item in enumerate(train['item'].unique().tolist()):
    item2id[item] = idx
train['user'] = train['user'].map(user2id)
train['item'] = train['item'].map(item2id)
test['user'] = test['user'].map(user2id)
test['item'] = test['item'].map(item2id)

# 将用户数据切分
train_data = []
test_data = []
test_target = {}
user_rating = {}
for uid in train['user'].unique().tolist():
    train_squence = train[train['user']==uid]['item'].tolist()
    user_rating[uid] = train_squence
    target = test[test['user']==uid]['item'].tolist()
    test_target[uid] = target
    for j in range(len(train_squence)-L-target_len):
        item_sq = train_squence[j:j+L]
        target_sq = train_squence[j+L:j+L+target_len]
        train_data.append([uid]+item_sq+target_sq)
    test_data.append([uid]+train_squence[-L:])

# 构造训练Dataloader
train_data = torch.LongTensor(train_data)
train_x = train_data[:,:1+L]
train_y = train_data[:,1+L:]
test_data = torch.LongTensor(test_data)
trainDataset = TensorDataset(train_x, train_y)
dataLoader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True)

In [5]:
def generate_neg_sample(batch_size,userid, user_rating, neg_num, num_item):
    neg_sample = []
    for uid in userid:
        result = []
        while len(result)<neg_num:
            neg_sampel_id = np.random.randint(0,num_item)
            while neg_sampel_id in user_rating[uid]:
                neg_sampel_id = np.random.randint(0,num_item)
            result.append(neg_sampel_id)
        neg_sample.append(result)
    return torch.LongTensor(neg_sample)

In [6]:
class Caser(nn.Module):
    def __init__(self, L, target_len, num_user,num_item,batch_size,outputs_h=16, outputs_v=4,embed_dimension=50):
        super(Caser, self).__init__()
        self.L = L
        self.target_len = target_len
        self.embed_dimension = embed_dimension
        self.batch_size = batch_size
        self.outputs_h = outputs_h
        self.outputs_v = outputs_v
        self.num_item = num_item
        # embedding matrix
        self.user_embedding = nn.Embedding(num_user, embed_dimension)
        self.item_embedding = nn.Embedding(num_item, embed_dimension)

        # horizontal convolution and pooling
        self.conv_pools = nn.ModuleList(nn.Sequential(
            nn.Conv2d(1, outputs_h, kernel_size=(k, embed_dimension)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(L-k+1, 1))) for k in range(1,L+1))

        # vertical convolution
        self.vertical_conv = nn.Conv2d(1, outputs_v, kernel_size=(L, 1))
        self.dropout = nn.Dropout(0.5)
        # full conect layer
        self.linear1 = nn.Linear(L*outputs_h+embed_dimension*outputs_v, embed_dimension)
        
        # full conect layer user item
        self.weights = nn.Embedding(num_item,embed_dimension+embed_dimension)
        self.bias = nn.Embedding(num_item,1)
        
        # embedding init
        self.user_embedding.weight.data.normal_(0,1.0/self.user_embedding.embedding_dim)
        self.item_embedding.weight.data.normal_(0,1.0/self.item_embedding.embedding_dim)
        self.weights.weight.data.normal_(0,1.0/self.weights.embedding_dim)
        self.bias.weight.data.zero_()
        
    def forward(self, user, items, y=None, for_pred=False):
        user_embed = self.user_embedding(user)
        user_embed = user_embed.squeeze(1)
        item_embed = self.item_embedding(items)
        
        item_embed = item_embed.unsqueeze(1)
        # horizontal convolution and pooling
        item_x_h = [conv_pool(item_embed) for conv_pool in self.conv_pools]
        item_x_h = torch.cat(item_x_h, 1)
        # vertical convolution
        item_x_v = self.vertical_conv(item_embed)
        # concatenate item sequence vector
        item_x_h = item_x_h.view(-1,self.outputs_h*self.L)
        item_x_v = item_x_v.view(-1,self.outputs_v*self.embed_dimension)

        x_c = torch.cat([item_x_h, item_x_v], 1)
        x_c = self.dropout(x_c)
        x_c = F.relu(self.linear1(x_c))
        # concatenate user and item vector
        x_u_i = torch.cat([x_c, user_embed], 1)
        
        if y is None:
            y = torch.LongTensor(np.arange(self.num_item)).cuda()

        weight = self.weights(y)
        bias = self.bias(y)
            
        if for_pred == False:
            # projection target target_len + negative_len
            res = torch.baddbmm(bias, weight, x_u_i.unsqueeze(2)).squeeze()
        else:
            res = torch.mm(weight,torch.transpose(x_u_i,1,0)).squeeze()
        return res       

In [7]:
def metric_prec_recall(predict, target, k):
    pred = predict[:k]
    hits = len(set(pred).intersection(set(target)))
    precision = hits/len(pred)
    recall = hits/len(target)
    return precision, recall

def metric_map(predict, target, k):
    if len(predict)>k:
        predict = predict[:k]
    score = 0.0
    hits = 0
    for i, val in enumerate(predict):
        if val in target and val not in predict[:i]:
            hits += 1
            score += hits/(i+1.0)
    if not list(target):
        return 0.0
    return score/min(len(target), k)

def test(model, test_data, k):
    model.eval()
    prec = []
    recall = []
    maps = []
    
    userids = test_data[:,0].unsqueeze(1).cuda()
    item_squence = test_data[:,1:].cuda()
    for i in range(userids.size(0)):
        uid = userids[i,:].unsqueeze(1)
        predicts = model(uid, item_squence[i].unsqueeze(0), for_pred=True)
        predicts = torch.argsort(predicts, descending=True)[:k].cpu().numpy()
        target = test_target[i]
        precision, rec = metric_prec_recall(predicts, target, k)
        ap = metric_map(predicts, target, k)
        prec.append(precision)
        recall.append(rec)
        maps.append(ap)
    return np.mean(prec), np.mean(recall), np.mean(maps)


In [8]:
def train_model(model, data, test_data, batch_size, neg_num, num_item,user_rating, epochs, lr=0.003):
    optimizer = optim.Adam(model.parameters(), weight_decay=0.000001, lr=lr)
    model.train()
    for epoch in range(epochs):
        start = time.time()
        losses = []
        for idx, (train_x, train_y) in enumerate(dataLoader):
            userid = train_x[:,0]
            item_squence = train_x[:,1:].cuda()
            target_squence = train_y
            negative_sample = generate_neg_sample(batch_size, userid.numpy(), user_rating, neg_num, num_item)
            y = torch.cat([target_squence, negative_sample],1).cuda()
            userid = userid.unsqueeze(1).cuda()
            y_pred = model(userid, item_squence, y)
            y_pred_pos, y_pred_neg = torch.split(y_pred, [target_squence.size(1),negative_sample.size(1)], dim=1)
            postive_loss = -torch.mean(torch.log(F.sigmoid(y_pred_pos)))
            negative_loss = -torch.mean(torch.log(1- F.sigmoid(y_pred_neg)))
            optimizer.zero_grad()
            loss = postive_loss + negative_loss
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
        print("Epoch %d loss is %.3f and consume time is %.2f" %(epoch+1, np.mean(losses), (time.time()-start)))
        precision, recall, maps = test(model, test_data, k=10)
        print("Test precision %.3f recall %.3f map %.3f" %(precision, recall, maps))

In [14]:
num_user = len(train['user'].unique())
num_item = len(train['item'].unique())
caser = Caser(L, target_len, num_user,num_item,batch_size).cuda()
train_model(caser, dataLoader,test_data, batch_size, neg_num, num_item,user_rating, epochs)

/root/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 1 loss is 0.714 and consume time is 42.40
Test precision 0.076 recall 0.052 map 0.032
Epoch 2 loss is 0.465 and consume time is 40.92
Test precision 0.090 recall 0.064 map 0.040
Epoch 3 loss is 0.377 and consume time is 42.40
Test precision 0.101 recall 0.076 map 0.048
Epoch 4 loss is 0.332 and consume time is 44.26
Test precision 0.103 recall 0.078 map 0.047
Epoch 5 loss is 0.304 and consume time is 43.90
Test precision 0.102 recall 0.076 map 0.047
Epoch 6 loss is 0.287 and consume time is 43.92
Test precision 0.105 recall 0.078 map 0.050
Epoch 7 loss is 0.276 and consume time is 42.13
Test precision 0.101 recall 0.075 map 0.047
Epoch 8 loss is 0.267 and consume time is 41.65
Test precision 0.101 recall 0.076 map 0.048
Epoch 9 loss is 0.260 and consume time is 41.82
Test precision 0.101 recall 0.076 map 0.047
Epoch 10 loss is 0.255 and consume time is 42.94
Test precision 0.102 recall 0.075 map 0.047
Epoch 11 loss is 0.250 and consume time is 42.81
Test precision 0.102 recall 0.